In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#######################
__version__ = "1.0"
__date__ = "2016-04-19"
__modified_by__ = "Hrushikesh Dhumal"
####################################

In [2]:
import os
import string
import numpy as np
import pandas as pd
from utils import *

In [3]:
from bs4 import BeautifulSoup
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [4]:
SEED_VAL = 200;
WORK_DIR = os.getcwd();
YELP_DATA_CSV_DIR = os.path.join(WORK_DIR, "data", "csv")
YELP_DATA_WORD_2_VEC_MODEL_DIR = os.path.join(WORK_DIR, "data", "word2vec_model")
YELP_DATA_FEAT_ADD = os.path.join(WORK_DIR, "data", "pos_neg")
YELP_DATA_WORD_2_VEC_MODEL_DIR_FEAT_ADD = os.path.join(WORK_DIR, "data", "word2vec_model_feat_add")

In [5]:
make_sure_path_exists(YELP_DATA_WORD_2_VEC_MODEL_DIR)

In [6]:
read_filename = os.path.join(YELP_DATA_CSV_DIR, 'business_review_user_0_1Percent.csv')
df_data = pd.read_csv(read_filename, engine='c', encoding='utf-8')

In [7]:
df_data.review_text[1]

u"I've been incredibly weary when it comes to mechanics as I've had some bad experiences, but I'm convinced that Tony and his team at Oasis are one of the best in town.\r\n\r\nI've been taking my Jeep Liberty into them since I bought it in 2010 after being referred by a friend, and they have consistently gone above and beyond when it comes to customer service, fair pricing, and clear communication about the vehicle's problem and options for getting it fixed. There have been one or two occasions where I've even called them, explained the problem, and they were able to diagnose the problem over the phone and - because the issue was so minor - walk me through how to take care of it on my own.\r\n\r\nGreat company that I completely trust with my vehicle. I would most definitely recommend Oasis to anyone who needs a good mechanic."

In [8]:
def lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords_and_stem(s):
    s = remove_numbers_in_string(s)
    s = lowercase_remove_punctuation(s)
    s = remove_stopwords(s)
    token_list = nltk.word_tokenize(s)
    #token_list = filter_out_more_stopwords(token_list)
    token_list = stem_token_list(token_list)
    return token_list

In [9]:
print lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords_and_stem(df_data.review_text[1])

[u'ive', u'incred', u'weari', u'come', u'mechan', u'ive', u'bad', u'experi', u'im', u'convinc', u'toni', u'team', u'oasi', u'one', u'best', u'town', u'ive', u'take', u'jeep', u'liberti', u'sinc', u'bought', u'refer', u'friend', u'consist', u'gone', u'beyond', u'come', u'custom', u'servic', u'fair', u'price', u'clear', u'commun', u'vehicl', u'problem', u'option', u'get', u'fix', u'one', u'two', u'occas', u'ive', u'even', u'call', u'explain', u'problem', u'abl', u'diagnos', u'problem', u'phone', u'issu', u'minor', u'walk', u'take', u'care', u'great', u'compani', u'complet', u'trust', u'vehicl', u'would', u'definit', u'recommend', u'oasi', u'anyon', u'need', u'good', u'mechan']


In [10]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords_and_stem( raw_sentence))
    
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [11]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in df_data["review_text"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


In [12]:
print sentences[0]

[u'place', u'great', u'allow', u'eat', u'like', u'much', u'littl', u'would', u'like']


In [13]:
df_data['pos_words'] = ''
df_data['neg_words'] = ''
df_data['net_senti'] = ''
df_data['review_senti'] = ''

In [14]:
POS_WORDS_FILE = os.path.join(YELP_DATA_FEAT_ADD, 'positive-words.txt')
NEG_WORDS_FILE = os.path.join(YELP_DATA_FEAT_ADD, 'negative-words.txt')

In [15]:
with open(POS_WORDS_FILE) as f:
    pos_words1 = f.read().split()[213:]

with open(NEG_WORDS_FILE) as f:
    neg_words1 = f.read().split()[213:]    
    
print "First 5 positive words %s " % pos_words1[:5]
print "First 5 negative words %s" % neg_words1[:5]

print "Number of positive words %d" % len(pos_words1)

print "Number of negative words %d" % len(neg_words1)

all_words_with_sentiment = pos_words1 + neg_words1

print "Total number of words %d" % len(all_words_with_sentiment)

First 5 positive words ['a+', 'abound', 'abounds', 'abundance', 'abundant'] 
First 5 negative words ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable']
Number of positive words 2006
Number of negative words 4783
Total number of words 6789


In [16]:
def lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords(s):
    s = remove_numbers_in_string(s)
    s = lowercase_remove_punctuation(s)
    s = remove_stopwords(s)
    token_list = []
    token_list.append(nltk.word_tokenize(s))
    #token_list = filter_out_more_stopwords(token_list)
    #token_list = stem_token_list(token_list)
    
    return token_list

In [17]:
all_reviews = [] # Initialize an empty list of sentences for each review

print "Parsing sentences from training set"
for review in df_data["review_text"]:
    all_reviews += lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords(review)

print len(all_reviews)
print len(df_data)
print all_reviews[0]

Parsing sentences from training set
2225
2225
[u'place', u'great', u'allows', u'eat', u'like', u'much', u'little', u'would', u'like', u'love', u'place', u'frequent', u'often', u'great', u'large', u'parties']


In [18]:
i = 0
for sentence in all_reviews:
    num_positive = sum([r in pos_words1 for r in sentence])
    num_negative = sum([r in neg_words1 for r in sentence])
    net_sentiment = num_positive - num_negative
     
#   num_positive #adds number of positive words to the dataframe for each review
#   num_negative #adds number of negative words to the dataframe for each review
#   net_sentiment #adds total weight to the dataframe for each review
    
    
    df_data.set_value(i, 'pos_words', num_positive)
    df_data.set_value(i, 'neg_words', num_negative)
    df_data.set_value(i, 'net_senti', net_sentiment)
    

    if(net_sentiment > 0):
        #adds 1 to the dataframe for each review denoting a positive tendency of the review
        df_data.set_value(i, 'review_senti', 1)
    elif(net_sentiment < 0):
        #adds -1 to the dataframe for each review denoting a negative tendency of the review
         df_data.set_value(i, 'review_senti', -1)
    else:
        #adds 0 to the dataframe for each review denoting a neutral tendency of the review
         df_data.set_value(i, 'review_senti', 0)
    i = i + 1

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:4: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [19]:
print df_data.head(10)

              business_id                                business_categories  \
0  SK77jbqCpIv_lwwRkhq9_w                     [u'Mongolian', u'Restaurants']   
1  oGii4C9XqsG5m3Q-V8LfgA                    [u'Auto Repair', u'Automotive']   
2  uXu7hX_bTrr-48op0Ykkyw  [u'Burgers', u'Breakfast & Brunch', u'Restaura...   
3  4uGHPY-OpJN08CabtTAvNg                        [u'French', u'Restaurants']   
4  nDnbIOqoKSAG339g6D8OnA  [u'Active Life', u'Yoga', u'Fitness & Instruct...   
5  c8XlQvVNwKhH9BOHEObtNQ        [u'American (Traditional)', u'Restaurants']   
6  PXviRcHR1mqdH4vRc2LEAQ                       [u'Burgers', u'Restaurants']   
7  bn2HqLj_XpPG9UE3mAo5cg  [u'Burgers', u'Breakfast & Brunch', u'American...   
8  nm8-Bn7O3DUytb5RZw4V5A  [u'Food', u'Ice Cream & Frozen Yogurt', u'Fast...   
9  1w6ii_Mb64da6cAxqxKF2A  [u'Hotels & Travel', u'Transportation', u'Airp...   

                business_name  business_review_count  \
0               Genghis Grill                    257   
1     O

In [20]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import gensim, logging
from gensim.models import Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [21]:
# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 10   # Minimum word count                        
num_workers = 8       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
model_file = os.path.join(YELP_DATA_WORD_2_VEC_MODEL_DIR, "100features_10minwords_10context__0_1Percent")
print "Training model..."
if not os.path.isfile(model_file):
# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
    model = word2vec.Word2Vec(sentences, workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = context, sample = downsampling)
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)
    model.save(model_file)
else:
    print "Loading existing model"
    model = Word2Vec.load(model_file)

Training model...
Loading existing model


In [22]:
model.doesnt_match("amazing delightful bad".split())

'bad'

In [23]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [24]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in df_data["review_text"]:
    clean_train_reviews.append( lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords_and_stem( review))

%time trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

Review 0 of 2225
Review 1000 of 2225
Review 2000 of 2225
Wall time: 772 ms


In [25]:
# b = np.array(df_data.review_stars.copy())
# b[(b == 1) | (b == 2) | (b == 3)] = 0
# b[(b == 4) | (b == 5)] = 1

In [26]:
# unique, counts = np.unique(b, return_counts=True)

# print np.asarray((unique, counts)).T

In [27]:
# trainDataVecs = np.column_stack((trainDataVecs, b))

In [28]:
trainDataVecs = np.column_stack((trainDataVecs, df_data.pos_words))
trainDataVecs = np.column_stack((trainDataVecs, df_data.neg_words))
trainDataVecs = np.column_stack((trainDataVecs, df_data.net_senti))
trainDataVecs = np.column_stack((trainDataVecs, df_data.review_senti))

In [29]:
trainDataVecs.shape

(2225L, 104L)

In [30]:
word2vec_add_feature_matrix_file = os.path.join(YELP_DATA_WORD_2_VEC_MODEL_DIR_FEAT_ADD, "word2vec_add_feature_matrix_0_1Percent.csv")
np.savetxt(word2vec_feature_matrix_file, trainDataVecs, delimiter=",")

In [31]:
test = np.genfromtxt(
    word2vec_add_feature_matrix_file,           # file name
    delimiter=',')           # column delimiter                                

In [32]:
np.array_equal(trainDataVecs, test)

True